# Sistema de recomendação 

O tutorial de recomendação clássico usa o [conjunto de dados movielens] (https://grouplens.org/datasets/movielens/). É semelhante a usar o conjunto de dados íris ou MNIST para outros algoritmos. Vamos fazer um código para ter uma ideia de como tudo isso funciona!


Procurando mais conjuntos de dados? Confira: https://gist.github.com/entaroadun/1653794

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

Com a filtragem colaborativa, fazemos previsões (filtragem) sobre os interesses de um usuário, coletando preferências ou informações de gosto de muitos usuários (colaborando). A suposição subjacente é que, se um usuário A tem a mesma opinião que um usuário B sobre um problema, é mais provável que A tenha a opinião de B sobre um assunto diferente x do que ter a opinião sobre x de um usuário escolhido aleatoriamente.

A imagem abaixo (da Wikipedia) mostra um exemplo de filtragem colaborativa. No início, as pessoas avaliam itens diferentes (como vídeos, imagens, jogos). Em seguida, o sistema faz previsões sobre a classificação de um usuário para um item ainda não classificado. As novas previsões são construídas sobre as classificações existentes de outros usuários com classificações semelhantes às do usuário ativo. Na imagem, o sistema prevê que o usuário não gostará do vídeo.

<img src = https: //upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

A biblioteca Spark MLlib para Machine Learning fornece uma implementação de Filtragem Colaborativa usando Mínimos Quadrados Alternados. A implementação em MLlib tem estes parâmetros:

* numBlocks é o número de blocos usados para paralelizar a computação (definido como -1 para configuração automática).
* classificação é o número de fatores latentes no modelo.
* iterations é o número de iterações a serem executadas.
* lambda especifica o parâmetro de regularização no ALS.
* implicitPrefs especifica se deve ser usada a variante ALS de feedback explícito ou uma adaptada para dados de feedback implícitos.
* alfa é um parâmetro aplicável à variante de feedback implícito do ALS que governa a confiança da linha de base nas observações de preferência.

Vamos ver tudo isso em ação!

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [4]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [5]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



Podemos fazer uma divisão para avaliar o desempenho de nosso modelo, mas tenha em mente que é muito difícil saber de forma conclusiva o quão bem um sistema de recomendação está realmente funcionando para alguns tópicos. Especialmente se a subjetividade estiver envolvida, por exemplo, nem todo mundo que adora Star Wars vai adorar a jornada nas estrelas, embora um sistema de recomendação possa sugerir o contrário.

In [7]:
# Conjunto de dados menor, então usaremos 0,8 / 0,2
(training, test) = data.randomSplit([0.8, 0.2])

In [8]:
# Construir o modelo de recomendação usando ALS nos dados de treinamento
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Agora vamos ver como o modelo se saiu!

In [9]:
# Avalie o modelo calculando o RMSE nos dados de teste
predictions = model.transform(test)

In [10]:
predictions.show()

+-------+------+------+------------+
|movieId|rating|userId|  prediction|
+-------+------+------+------------+
|     31|   1.0|    26|   0.5064666|
|     85|   1.0|    13|   1.5420086|
|     85|   1.0|    23|   1.8684256|
|     85|   1.0|    25|   1.6629062|
|     85|   1.0|    29|   1.1261266|
|     65|   1.0|    16|   1.2183961|
|     65|   2.0|     3|   2.0900104|
|     65|   1.0|    19|   1.0170004|
|     53|   1.0|    23| 0.081142426|
|     53|   3.0|    14|   3.8325825|
|     78|   1.0|     1|  0.56496745|
|     78|   1.0|     4|  0.67932105|
|     34|   4.0|     2|   0.0833499|
|     81|   1.0|     6|    1.298579|
|     81|   1.0|    16|  -1.2940426|
|     81|   2.0|     9|   2.2675438|
|     81|   2.0|    29|   1.8942225|
|     28|   3.0|     1|   1.2952838|
|     28|   1.0|    14|0.0031218529|
|     76|   1.0|    20|   0.7271807|
+-------+------+------+------------+
only showing top 20 rows



In [11]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.560783677189561


O RMSE descreveu nosso erro em termos da coluna de classificação de estrelas.

Agora que temos o modelo, como você realmente forneceria uma recomendação a um usuário?

Da mesma forma que fizemos com os dados de teste! Por exemplo:

In [12]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [13]:
# O usuário teve 10 avaliações no conjunto de dados de teste
# Realisticamente, deve ser algum tipo de conjunto de suporte!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     13|    11|
|     16|    11|
|     30|    11|
|     35|    11|
|     43|    11|
|     51|    11|
|     89|    11|
|     90|    11|
+-------+------+



In [14]:
reccomendations = model.transform(single_user)

In [16]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     30|    11|  3.263875|
|     35|    11| 3.1437485|
|     51|    11| 1.2200018|
|     16|    11| 1.2183044|
|     43|    11| 0.3729321|
|     90|    11|0.20518604|
|     13|    11|-0.7660935|
|     89|    11|-0.7715881|
+-------+------+----------+

